In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

user = "root"
password = os.getenv("MYSQL_PASSWORD")
host = "127.0.0.1"
port = 3306
database = "bi_sales_db"

password_quoted = quote_plus(password) if password is not None else ""
connection_url = f"mysql+mysqlconnector://{user}:{password_quoted}@{host}:{port}/{database}"

engine = create_engine(connection_url)

pd.read_sql(text("SHOW TABLES"), engine)

,Tables_in_bi_sales_db
0,customers
1,products
2,sales


### NOTE: 
On my local machine I set MYSQL_PASSWORD in the environment (via `export MYSQL_PASSWORD='...'` in Terminal) so the password is not stored in this notebook.

In [2]:
import random
from datetime import datetime, timedelta

# customers
customer_rows = []
regions = ["North", "South", "East", "West"]

for customer_id in range(1, 51):  # 50 customers
    name = f"Customer_{customer_id}"
    region = random.choice(regions)
    customer_rows.append({"customer_id": customer_id,
                          "customer_name": name,
                          "region": region})

customers_df = pd.DataFrame(customer_rows)

product_rows = []
categories = ["Electronics", "Home", "Clothing", "Food"]

# products
for product_id in range(1, 21):  
    name = f"Product_{product_id}"
    category = random.choice(categories)
    price = round(random.uniform(5, 500), 2)
    product_rows.append({"product_id": product_id,
                         "product_name": name,
                         "category": category,
                         "price": price})

products_df = pd.DataFrame(product_rows)

#sales
sale_rows = []
start_date = datetime(2023, 1, 1)

def random_date_2023():
    days_offset = random.randint(0, 364)
    return (start_date + timedelta(days=days_offset)).date()

for sale_id in range(1, 1001):  # 1000 sales
    cust_id = random.randint(1, 50)   # must match customers
    prod_id = random.randint(1, 20)   # must match products
    sale_date = random_date_2023()
    amount = round(random.uniform(10, 1000), 2)

    sale_rows.append({"sale_id": sale_id,
                      "customer_id": cust_id,
                      "product_id": prod_id,
                      "sale_date": sale_date,
                      "sale_amount": amount})

for row in random.sample(sale_rows, 20):
    row["sale_amount"] = 0.0
for row in random.sample(sale_rows, 20):
    row["sale_amount"] = None

sales_df = pd.DataFrame(sale_rows)

customers_df.head(), products_df.head(), sales_df.head()

(   customer_id customer_name region
 0            1    Customer_1  North
 1            2    Customer_2   East
 2            3    Customer_3   West
 3            4    Customer_4  North
 4            5    Customer_5  South,
    product_id product_name     category   price
 0           1    Product_1         Food  201.77
 1           2    Product_2         Food   86.37
 2           3    Product_3     Clothing  365.96
 3           4    Product_4         Home  286.67
 4           5    Product_5  Electronics  111.47,
    sale_id  customer_id  product_id   sale_date  sale_amount
 0        1           15          11  2023-04-13       312.83
 1        2            1           3  2023-01-05         0.00
 2        3           49          20  2023-11-19       263.02
 3        4           12          19  2023-12-17       121.40
 4        5           32           9  2023-03-04       316.70)

In [3]:
customers_df.to_csv('sample_customers.csv', index=False)
products_df.to_csv('sample_products.csv', index=False)
sales_df.to_csv('sample_sales.csv', index=False)

### (Optional) Exported data to CSV for Tableau

I used it only to export the generated DataFrames to CSV so I could
build a dashboard in Tableau:

- `customers_df` → `sample_customers.csv`
- `products_df` → `sample_products.csv`
- `sales_df` → `sample_sales.csv`

These CSV files were then used as data sources in Tableau to create the
*Sales by Category*, *Sales by Month*, and *Sales by Region* visualizations.

In [4]:
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS sales"))
    conn.execute(text("DROP TABLE IF EXISTS customers"))
    conn.execute(text("DROP TABLE IF EXISTS products"))

customers_df.to_sql("customers", engine, index=False, if_exists="replace")
products_df.to_sql("products", engine, index=False, if_exists="replace")
sales_df.to_sql("sales", engine, index=False, if_exists="replace")

print(pd.read_sql(text("SELECT COUNT(*) AS n FROM customers"), engine))
print(pd.read_sql(text("SELECT COUNT(*) AS n FROM products"), engine))
print(pd.read_sql(text("SELECT COUNT(*) AS n FROM sales"), engine))

    n
0  50
    n
0  20
      n
0  1000


In [5]:
# join sales + customers + products

join_sql = """
SELECT
    s.sale_id,
    s.sale_date,
    s.sale_amount,
    c.customer_id,
    c.customer_name,
    c.region,
    p.product_id,
    p.product_name,
    p.category,
    p.price
FROM sales AS s
JOIN customers AS c ON s.customer_id = c.customer_id
JOIN products  AS p ON s.product_id  = p.product_id
"""

joined_df = pd.read_sql(text(join_sql), engine)

print("Joined data (first 5 rows):")
joined_df.head()

Joined data (first 5 rows):


,sale_id,sale_date,sale_amount,customer_id,customer_name,region,product_id,product_name,category,price
0,1,2023-04-13,312.83,15,Customer_15,South,11,Product_11,Electronics,403.21
1,2,2023-01-05,0.00,1,Customer_1,North,3,Product_3,Clothing,365.96
2,3,2023-11-19,263.02,49,Customer_49,West,20,Product_20,Food,135.45
3,4,2023-12-17,121.40,12,Customer_12,South,19,Product_19,Food,451.98
4,5,2023-03-04,316.70,32,Customer_32,North,9,Product_9,Clothing,244.76


In [6]:
data = joined_df.copy()

data = data[data["sale_amount"].notna()]     
data = data[data["sale_amount"] > 0]         

data["sale_date"] = pd.to_datetime(data["sale_date"])
data["year"] = data["sale_date"].dt.year
data["month"] = data["sale_date"].dt.to_period("M").astype(str)  

# total sales amount for each product category
sales_by_category = (
    data
    .groupby("category")["sale_amount"]
    .sum()
    .reset_index()
    .rename(columns={"sale_amount": "total_sales"})
)

print("Total sales by category:")
display(sales_by_category)

# total sales amount and average sale amount by region
sales_by_region = (
    data
    .groupby("region")["sale_amount"]
    .agg(["sum", "mean"])
    .reset_index()
)

sales_by_region = sales_by_region.rename(
    columns={"sum": "total_sales", "mean": "avg_sale_amount"}
)

print("Total and average sale amount by region:")
display(sales_by_region)

# monthly sales trend (total sales per month)
monthly_sales = (
    data
    .groupby("month")["sale_amount"]
    .sum()
    .reset_index()
    .rename(columns={"sale_amount": "total_sales"})
    .sort_values("month")
)

print("Monthly sales trend (total sales per month):")
display(monthly_sales)

Total sales by category:


,category,total_sales
0,Clothing,136574.71
1,Electronics,118413.05
2,Food,112879.94
3,Home,121602.63


Total and average sale amount by region:


,region,total_sales,avg_sale_amount
0,East,106746.12,494.195000
1,North,92798.73,518.428659
2,South,128788.40,509.045059
3,West,161137.08,516.465000


Monthly sales trend (total sales per month):


,month,total_sales
0,2023-01,50170.51
1,2023-02,42343.36
2,2023-03,35406.25
3,2023-04,36785.57
4,2023-05,33944.32
5,2023-06,33436.48
6,2023-07,43160.10
7,2023-08,45265.24
8,2023-09,42088.69
9,2023-10,39742.89


In [7]:
sales_by_category.to_csv("sales_by_category.csv", index=False)
sales_by_region.to_csv("sales_by_region.csv", index=False)
monthly_sales.to_csv("monthly_sales.csv", index=False)